# 1. 개발 환경 설정

### 1.1 필수 라이브러리 설치하기

In [1]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 597.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 14.9

In [2]:
# prompt: i want to print out the versions only

!nvidia-smi
!echo "bitsandbytes version:"
!pip show bitsandbytes
!echo "peft version:"
!pip show peft
!echo "trl version:"
!pip show trl
!echo "accelerate version:"
!pip show accelerate
!echo "datasets version:"
!pip show datasets
!echo "transformers version:"
!pip show transformers

Wed Aug 14 10:20:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# 구글 드라이브 마운트

from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

Mounted at /content/drive


### 1.2 Import modules

In [4]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm
from datasets import DatasetDict
from accelerate import Accelerator

# 2. Dataset 생성 및 준비

In [6]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/2024인하대/open.zip"

Archive:  /content/drive/MyDrive/2024인하대/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
file_path = '/content/train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

train_data.head(2)

,id,context,question,answer
0,TRAIN_10090,닛산이 새로운 상표권을 등록하며 시장 관계자들의 이목을 집중시켰다.\n최근 알려진 ...,닛산이 새롭게 등록한 상표권의 이름은,I-파워
1,TRAIN_06253,공정거래위원회가 사내급식 일감을 삼성웰스토리에 전부 몰아준 혐의로 삼성그룹에 과징금...,사내급식을 제공하는 현대그린푸드는 어떤 그룹에 속해 있는가,현대백화점그룹


In [8]:
train_data = train_data[train_data['context'].apply(len) <= 1300]

In [9]:
train_data

,id,context,question,answer
0,TRAIN_10090,닛산이 새로운 상표권을 등록하며 시장 관계자들의 이목을 집중시켰다.\n최근 알려진 ...,닛산이 새롭게 등록한 상표권의 이름은,I-파워
1,TRAIN_06253,공정거래위원회가 사내급식 일감을 삼성웰스토리에 전부 몰아준 혐의로 삼성그룹에 과징금...,사내급식을 제공하는 현대그린푸드는 어떤 그룹에 속해 있는가,현대백화점그룹
3,TRAIN_11278,럭셔리 비즈니스 라이프 스타일 브랜드 몽블랑(Montblanc)이 직관적 디스플레이...,몽블랑에서 만든 스마트워치는 무슨 소재를 이용해서 만든 거야,재생 알루미늄
4,TRAIN_15111,설에 명절 상여금을 지급하겠다는 중소기업들이 지난해보다 크게 줄었다. 경영난 원인으...,중소기업 설 상여금에 관련된 리서치에서 지급예정이라고 한 데는 얼마나 돼,36.70%
5,TRAIN_02205,NH농협 무주군지부(지부장 오영석)가 코로나19 여파로 경영 어려움에 처한 관내 화...,화초 농사를 짓는 농민들의 고충을 덜어주기 위해 농협 무주군지부가 구입해서 나눠준 ...,씨클라멘 화분 330여개와 장미 500송이 등
...,...,...,...,...
33710,TRAIN_19669,이커머스 전문기업 코리아센터는 지난 20일 부산시 남구청에서 '저소득가구 아동 지원...,저소득층 아동을 위해 어느 기업에서 성금을 기탁했어,코리아센터
33712,TRAIN_20768,전라북도와 전북바이오융합진흥원(원장 김동수)이 고령자 건강관리를 위한 영양관리 스마...,스마트 앱 시스템의 효과를 알아보기 위해 모니터링 한 대상은 누구야,두억마을 고령자
33713,TRAIN_22978,"주가 3,000시대를 맞은 동학개미들의 화력이 심상치 않다. 개인 투자자들은 최근 ...",12일엔 얼마에 코스피 지수가 마무리된 거야,"3,125.95"
33714,TRAIN_31460,한국지역난방공사(사장 황창화)는 지난해 12월 ‘난방 ON 서비스’ 시범사업 시행 ...,12월부터 한국지역난방공사가 무료로 진행하는 방문 남방 검사가 뭐야,난방 ON 서비스


In [10]:
dataset = Dataset.from_pandas(train_data)
dataset

Dataset({
    features: ['id', 'context', 'question', 'answer', '__index_level_0__'],
    num_rows: 28574
})

In [11]:
dataset[0]

{'id': 'TRAIN_10090',
 'context': '닛산이 새로운 상표권을 등록하며 시장 관계자들의 이목을 집중시켰다.\n최근 알려진 바에 따르면 닛산이 I-파워(I-Power)라고 명명된 새로운 상표권을 등록한 것으로 알려졌다.\n상표권에 있어서 그 목적과 개념, 그리고 그 사용성 등에 있어 구체적인 설명이 없는 만큼 I-파워의 목적은 명확하지 않다.\n그러나 닛산이 이미 e-파워(e-Power)로 명명한 상표권을 사용하고 있는 만큼 I-파워 역시 비슷한 개념으로 사용될 것으로 예상되고 있다.\n닛산 e-파워가 전동화 모델에 적용되는 만큼 I-파워 역시 전동화 모델에 적용될 것으로 예상되고 있다.\n특히 인피니티가 전동화 기조, 미래 비전에 대한 컨셉을 품고 있는 Qs 및 QX 인스퍼레이션(Inspiration) 역시 I를 앞세운 만큼 전동화 모델을 위한 상표권으로 분석되고 있다.\n다만 닛산은 아직 I-파워에 대한 여러 가설 및 질문에 대해 구체적인 발표는 따로 한 부분이 없는 상태다.\n과연 닛산이 새롭게 등록한 I-파워는 과연 어떤 모습으로 등장할지 그 귀추를 주목해본다.',
 'question': '닛산이 새롭게 등록한 상표권의 이름은',
 'answer': 'I-파워',
 '__index_level_0__': 0}

# 3. Gemma 파인튜닝

In [12]:
# !nvidia-smi
import subprocess

# GPU 상태 확인
def check_gpu():
    try:
        result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
        print(result.stdout.decode('utf-8'))
    except FileNotFoundError:
        print("nvidia-smi not found, ensure you have a GPU instance.")

check_gpu()

Wed Aug 14 10:21:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### 3.1 학습용 프롬프트 조정

In [13]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['context'])):
        prompt_list.append(r"""<bos><start_of_turn>user
너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.:

{}

Question:
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['context'][i], example['question'][i], example['answer'][i]))
    return prompt_list

In [14]:
train_data = dataset
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해.:

닛산이 새로운 상표권을 등록하며 시장 관계자들의 이목을 집중시켰다.
최근 알려진 바에 따르면 닛산이 I-파워(I-Power)라고 명명된 새로운 상표권을 등록한 것으로 알려졌다.
상표권에 있어서 그 목적과 개념, 그리고 그 사용성 등에 있어 구체적인 설명이 없는 만큼 I-파워의 목적은 명확하지 않다.
그러나 닛산이 이미 e-파워(e-Power)로 명명한 상표권을 사용하고 있는 만큼 I-파워 역시 비슷한 개념으로 사용될 것으로 예상되고 있다.
닛산 e-파워가 전동화 모델에 적용되는 만큼 I-파워 역시 전동화 모델에 적용될 것으로 예상되고 있다.
특히 인피니티가 전동화 기조, 미래 비전에 대한 컨셉을 품고 있는 Qs 및 QX 인스퍼레이션(Inspiration) 역시 I를 앞세운 만큼 전동화 모델을 위한 상표권으로 분석되고 있다.
다만 닛산은 아직 I-파워에 대한 여러 가설 및 질문에 대해 구체적인 발표는 따로 한 부분이 없는 상태다.
과연 닛산이 새롭게 등록한 I-파워는 과연 어떤 모습으로 등장할지 그 귀추를 주목해본다.

Question:
닛산이 새롭게 등록한 상표권의 이름은<end_of_turn>
<start_of_turn>model
I-파워<end_of_turn><eos>


### 3.2 QLoRA 설정

In [15]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [16]:
BASE_MODEL = "rtzr/ko-gemma-2-9b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 3.3 Trainer 실행

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=None,
    args=TrainingArguments(
        output_dir="gemma2_results",
        # 빠른 테스트를 위해 epoch 값 1로 고정
        num_train_epochs = 1,
        max_steps=-1,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        warmup_ratio=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1000,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/28574 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss


### 3.4 Finetuned Model 저장

In [ ]:
ADAPTER_MODEL = "/content/drive/MyDrive/gemma2_lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)